In [39]:
import os
import json
import pandas as pd 
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
key=os.getenv("OPENAI_API_KEY")

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
llm=ChatOpenAI(openai_api_key=key,model_name='gpt-3.5-turbo',temperature=0.6)

c:\Users\USER\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain

In [59]:
response_json={
    1:{"mcq":"question will be here",
     "options": {
            'a':"first option",
            'b':"second option",
            'c':"third option",
            'd':"fourth option"
        },
    
    "correct":"correct option"
    },
    2:{"mcq":"question will be here",
     "options": {
            'a':"first option",
            'b':"second option",
            'c':"third option",
            'd':"fourth option"
        },
    
    "correct":"correct option"
    }
}

In [60]:
template1="{text}.you have a text.now your task is to generate {number} mcq questions from this above text for {subject} student in {tone} tone.make sure that no mcq question is repeated and the questions are only from the above text.for generating mcq questions you have to follow the below response_json format.{response_json}"

In [61]:
quiz_generation_prompt1=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=template1
)

In [62]:
quiz=LLMChain(llm=llm,prompt=quiz_generation_prompt1,output_key="quiz")

In [63]:
template2="You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities. Quiz_MCQs:{quiz}"

In [64]:
correct_answer_generation_prompt2=PromptTemplate(
    input_variables=["subject","quiz"],
    template=template2
)

In [65]:
answer=LLMChain(llm=llm,prompt=correct_answer_generation_prompt2,output_key="review")

In [66]:
mcq_chain=SequentialChain(chains=[quiz,answer],input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz","review"])

In [67]:
path=r'C:\Users\USER\mcqgen\data.txt'
with open(path, 'r') as file:
    text=file.read()

In [68]:
mcq=mcq_chain({"text":text,"number":7,"subject":"Data Science","tone":"simple","response_json":response_json})
quiz_table=mcq['quiz']

In [69]:
df=pd.DataFrame(eval(quiz_table))
my_quiz_table=df.transpose()
print(my_quiz_table)

                                                 mcq  \
1  Which type of data analysis focuses on underst...   
2  What type of data analysis uses historical dat...   
3  What type of analysis not only predicts what i...   
4  Which technique is characterized by data visua...   
5  What technique involves computers being traine...   
6  Which type of analysis involves deep-dive or d...   
7  What technique uses graph analysis, simulation...   

                                             options correct  
1  {'a': 'Descriptive analysis', 'b': 'Diagnostic...       b  
2  {'a': 'Descriptive analysis', 'b': 'Diagnostic...       c  
3  {'a': 'Descriptive analysis', 'b': 'Diagnostic...       d  
4  {'a': 'Drill-down', 'b': 'Data mining', 'c': '...       d  
5  {'a': 'Machine learning', 'b': 'Forecasting', ...       a  
6  {'a': 'Descriptive analysis', 'b': 'Diagnostic...       b  
7  {'a': 'Data mining', 'b': 'Prescriptive analyt...       b  


In [70]:
my_quiz_table.to_csv("Data_Science_Quiz.csv")